In [1]:
%matplotlib widget
from dorna_vision import util, draw
import matplotlib.pyplot as plt
from camera import Camera
from dorna2 import Dorna, Kinematic
import cv2
import numpy as np
import itertools
from scipy.optimize import minimize


In [ ]:
"""
System parameters
"""
# robot
robot_ip = "192.168.254.30"# robot ip address 
robot_model = "dorna_ta"

# aruco
aruco_id = 17
marker_length=24 # mm
dictionary="DICT_4X4_100"
refine="CORNER_REFINE_APRILTAG"
coordinate="CCW"
subpix=False

# search area
deviations = np.array(list(itertools.product([-10, 10], [-10, 10], [-10, 10], [-10, 10], [-10, 10],[0])))

# collected data
collected_data = []

In [4]:
"""
helper functions
"""
# euler to transformation matrix
def Euler_matrix(abg,xyz):
    cv0 = np.cos(abg[0])
    sv0 = np.sin(abg[0])
    cv1 = np.cos(abg[1])
    sv1 = np.sin(abg[1])
    cv2 = np.cos(abg[2])
    sv2 = np.sin(abg[2])
    return np.matrix([
        [cv1* cv0   , sv2*sv1*cv0 - cv2*sv0 , cv2*sv1*cv0 - sv2*sv0 , xyz[0]  ],
        [cv1 * sv0  , sv2*sv1*sv0 + cv2*cv0 , cv2*sv1*sv0 + sv2*cv0 , xyz[1]  ],
        [-sv1       , sv2*cv1               , cv2*cv1               , xyz[2]  ],
        [0,0,0,1]])


def likelihood(p, joints_list, t_target_2_cam_list, kinematic, centroid=None):
    T_cam_2_j4 = Euler_matrix([p[3],p[4],p[5]],[p[0],p[1],p[2]])
    v =[]
    for i in range(len(joints)):
        g = np.matmul(np.matmul(kinematic.Ti_r_world(i=5, joint=joint_list[i]),np.matrix(T_cam_2_j4)), np.vstack((t_target_2_cam_list[i], np.array([[1]]))))
        v.append([tmp[0,0],tmp[1,0],tmp[2,0]])

    if centroid == None:
        # pass
        
    distance = np.linalg.norm([np.linalg.norm(g-centroid) for g in v])
    return np.linalg.norm(distance)


def minimizer(joints, t_target_2_cam_list, kinematic):

    R_j4_2_base_list = []
    t_j4_2_base_list = []

    for j in joints:
        T_j4_2_base = kinematic.Ti_r_world(i=5, joint=j)
        R_j4_2_base_list.append(np.array(T_j4_2_base[:3, :3])) 
        t_j4_2_base_list.append(np.array(T_j4_2_base[:3, 3]) )

    data_t = [np.array(d) for d in t_target_2_cam_list]
    data_R = []

    for r in R_target_2_cam_list:
        rotation_matrix = np.zeros(shape=(3,3))
        cv2.Rodrigues(r, rotation_matrix)
        data_R.append(np.array(rotation_matrix))

    #T_cam_2_j4 = np.eye(4)

    f = minimize(likelihood, [0,0,0,0,0,0])
    
    T_cam_2_j4 = Euler_matrix([f.x[3],f.x[4],f.x[5]],[f.x[0],f.x[1],f.x[2]])
    """
    T_cam_2_j4 = np.matrix([[-1,  0,  0,  f.x[0]],
                            [0, -1,  0,  f.x[1]],
                            [ 0, 0,  1,  f.x[2]],
                            [ 0,  0,  0,  1]])
    """
    print("final error: ", f.fun)
    return T_cam_2_j4


def dorna_ta_eye_in_hand_embeded_camera(data, robot, kinematic, camera, joint_deviation, aruco_id, aruco_length, aruco_dic, aruco_refine, aruco_subpix, aruco_coordinate):
    # record t_target_2_base
    touching = input("The robot is touching the aruco marker (Y or N):")
    touching = touching.lower()
    
    if touching == "y":
        xyzabc = robot.get_all_pose()
        
        #go up
        robot.set_motor(1)
        robot.lmove(rel=1, x=-40, y=30, z=120, vel=100, accel=1000, jerk=2000)
        robot.sleep(1)
        
    
    # initial joint
    initial_joint = robot.get_all_joint()
    target_joint_list = [(initial_joint +  joint).tolist() for joint in joint_deviation]
    
    # Generate all possible lists of size 5    
    for joint in target_joint_list:
        for i in rage(1):
            robot.jmove(rel=0, vel=50, accel=800, jerk=1000, j0=joint[0], j1=joint[1], j2=joint[2], j3=joint[3], j4=joint[4], j5=joint[5])
            robot.sleep(1)
    
            # capture image
            depth_frame, _, _, _, _, color_img, depth_int, _, _= camera.get_all()
    
            # joint
            joint = robot.get_all_joint()
            
            # search_id pose: [[id, corner, rvec, tvec] for id, corner, rvec, tvec in zip(aruco_id, aruco_corner, rvecs, tvecs)]
            aruco_data = util.find_aruco(color_img, camera.camera_matrix(depth_int), camera.dist_coeffs(depth_int), dictionary=aruco_dic, marker_length=aruco_length, refine=aruco_refine, subpix=aruco_subpix, coordinate=aruco_coordinate)
            
            # find aruco_id
            pxl_target_2_cam = []
            for val in retval:
                if val[0] == aruco_id:
                    pxl_target_2_cam = val[1]
                    break
            
            # aruco not found
            if not pxl_target_2_cam:
                break

            # target to camera
            xyz_target_2_cam, _ = camera.xyz(pxl_target_2_cam, depth_frame, depth_int)

            # append collect data
            data.append({"joint": joint, "t_target_2_cam": xyz_target_2_cam.reshape(-1, 1), "t_target_2_base": xyz_target_2_base.reshape(-1, 1)})

            # draw aruco
            draw.draw_aruco(color_img, aruco_data, camera.camera_matrix(depth_int), camera.dist_coeffs(depth_int))
        
        # Display the image
        fig, ax = plt.subplots(frameon=False)
        ax.imshow(cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB))
        plt.show()

    return data


2.0

In [3]:
# camera
camera = Camera()
print("Camera connected: ", camera.connect())

# Robot
robot = Dorna()
print("Robot connected: ", robot.connect(robot_ip))

# kinematics
kinematic = Kinematic(robot_model)

# update collected data
collected_data += dorna_ta_eye_in_hand_embeded_camera(collected_data, robot, kinematic, camera, charuco_board, joint_list, ground_truth, joint_calibration, file_path)  

T_cam_2_j4 = minimizer( joints = data["joints"], 
                        R_target_2_cam_list = data["R_target_2_cam_list"], 
                        t_target_2_cam_list = data["t_target_2_cam_list"], 
                        kinematic = kinematic, 
                        ground_truth = ground_truth, 
                        use_rotation = False,
                        joint_calibration=joint_calibration)

# close the connections
camera.close()
robot.close()

print(T_cam_2_j4)


Camera connected:  True
Robot connected:  True


In [9]:
upper("y")

NameError: name 'upper' is not defined

In [10]:
"y".lower()

'y'